In [184]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

import unicodedata
from unidecode import unidecode
import time
import os

import pandas as pd
import numpy as np
pd.options.display.max_rows = 85
pd.options.display.max_columns = 200

## Webscrapping

In [147]:
# Scrapping Statistics
def scrape_stats_to_df(year):
    url1 = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html".format(year)
    html1 = urlopen(url1)
    soup1 = BeautifulSoup(html1)
    # use getText()to extract the text we need into a list
    headers1 = [th.getText() for th in soup1.findAll('tr', limit=1)[0].findAll('th')]
    # exclude the first column since it's 'Rank'.
    headers1 = headers1[1:]
    # avoid the first header row to get all rows
    rows1 = soup1.findAll('tr')[1:]
    player_stats = [[td.getText() for td in rows1[i].findAll('td')] 
                    for i in range(len(rows1))]
    df_stats = pd.DataFrame(player_stats, columns=headers1)
    
    # Drop null values
    df_stats.dropna(inplace=True)
    # Drop duplicate entries for players that got traded mid-way through the season.
    df_stats.drop_duplicates('Player', keep='first', inplace=True)
    df_stats = df_stats.reset_index(drop=True)
    
    # Converted Special Characters to unicode and removed bad words and characters.
    s = df_stats['Player']
    coded = []
    for i in s:
        coded.append(unidecode(u'{}'.format(i)))
    bad_chars = [".", "'","*"]
    string_list = []
    for word in coded:
        string = word.replace(' Jr.', '')
        string = string.replace(' III', '')
        string = ''.join(i for i in string if not i in bad_chars) 
        string_list.append(string)
    df_stats['Player'] = pd.Series(string_list)
    return df_stats

In [148]:
# Scrapping Salary
def scrape_salary_to_df(year):
    url2 = "https://hoopshype.com/salaries/players/{}/".format(str(year-1)+'-'+str(year))
    html2 = urlopen(url2)
    soup2 = BeautifulSoup(html2)
    # use getText()to extract the text we need into a list
    headers2 = [(th.getText().strip()) for th in soup2.findAll('tr', limit=1)[0].findAll('td')]
    # must name the first column since it is ''.
    headers2[0] = 'Rank'
    rows2 = soup2.findAll('tr')[1:]
    player_salary = [[td.getText().strip() for td in rows2[i].findAll('td')]
                for i in range(len(rows2))]
    
    ## Make Dataframe
    df_salary = pd.DataFrame(player_salary, columns = headers2)  
    
    # Cleaning Salary Name
    names = df_salary['Player']
    coded_names = []
    for i in names:
        coded_names.append(unidecode(u'{}'.format(i)))
    bad_chars = [".", "'","*"]
    salary_list = []
    for word in coded_names:
        string = word.replace(' Jr', '')
        string = string.replace(' III', '')
        string = ''.join(i for i in string if not i in bad_chars) 
        salary_list.append(string)
    df_salary['Player'] = pd.Series(salary_list)
    return df_salary

In [149]:
salary_cap ={'1985': '$3,600,000',
'1986': '$4,233,000',
'1987': '$4,945,000',
'1988': '$6,164,000',
'1989': '$7,232,000',
'1990': '$9,802,000',
'1991': '$11,871,000',
'1992': '$12,500,000',
'1993': '$14,000,000',
'1994': '$15,175,000',
'1995': '$15,964,000',
'1996': '$23,000,000',
'1997': '$24,363,000',
'1998': '$26,900,000',
'1999': '$30,000,000',
'2000': '$34,000,000',
'2001': '$35,500,000',
'2002': '$42,500,000',
'2003': '$40,271,000',
'2004': '$43,840,000',
'2005': '$43,870,000',
'2006': '$49,500,000',
'2007': '$53,135,000',
'2008': '$55,630,000',
'2009': '$58,680,000',
'2010': '$57,700,000',
'2011': '$58,044,000',
'2012': '$58,044,000',
'2013': '$58,044,000',
'2014': '$58,679,000',
'2015': '$63,065,000',
'2016': '$70,000,000',
'2017': '$94,143,000',
'2018': '$99,093,000',
'2019': '$101,869,000',
'2020': '$109,140,000'}

df_salarycap = pd.DataFrame(salary_cap.items(), columns=['Year', 'SalaryCap'])

In [150]:
total_years = np.arange(2000, 2020)

In [151]:
total_years

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [152]:
### DO NOT RUN IF YOU ALREADY HAVE THE DATA
#Run web scrapping
all_data = []
for year in total_years:
    ## Make Dataframe of Basketball Stats
    df_stats = scrape_stats_to_df(year)
    
    ## Make Dataframe of Player Salary
    df_salary = scrape_salary_to_df(year)
    
    ## Merge Stats and Salary Dataset
    df_ss = pd.merge(df_stats, df_salary, on='Player', how='left')
    # Rename Salary Column
    df_ss.rename(columns={df_ss.columns[30]: 'Salary'}, inplace=True)
    # Drop Rank and Adjusted Salary column
    df_ss.drop(df_ss.columns[[29,31]], axis=1, inplace=True)
    
    ## Create Id column
    # 1. Remove spaces on player name
    n = df_ss['Player']
    n_list = []
    for word in n:
        string = word.replace(' ', '')
        n_list.append(string)    
    df_ss['Player'] = pd.Series(n_list)  
    # 2. Create Year Column
    df_ss['Year'] = str(year)
    # 3. Create id column
    df_ss['id'] = df_ss['Player'] + df_ss['Year']
    
    ## Merge dataset with Salarycap
    df_ssc = pd.merge(df_ss, df_salarycap, on='Year', how='left')
    all_data.append(df_ssc)
    
    ## Give time to rest
    time.sleep(0.2)
    if year%5 == 0:
        time.sleep(5)

In [153]:
df = pd.concat(all_data)

In [154]:
df = df.reset_index(drop=True)

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9310 entries, 0 to 9309
Data columns (total 33 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Player     9310 non-null   object
 1   Pos        9310 non-null   object
 2   Age        9310 non-null   object
 3   Tm         9310 non-null   object
 4   G          9310 non-null   object
 5   GS         9310 non-null   object
 6   MP         9310 non-null   object
 7   FG         9310 non-null   object
 8   FGA        9310 non-null   object
 9   FG%        9310 non-null   object
 10  3P         9310 non-null   object
 11  3PA        9310 non-null   object
 12  3P%        9310 non-null   object
 13  2P         9310 non-null   object
 14  2PA        9310 non-null   object
 15  2P%        9310 non-null   object
 16  eFG%       9310 non-null   object
 17  FT         9310 non-null   object
 18  FTA        9310 non-null   object
 19  FT%        9310 non-null   object
 20  ORB        9310 non-null   obj

In [156]:
df.isna().sum()

Player         0
Pos            0
Age            0
Tm             0
G              0
GS             0
MP             0
FG             0
FGA            0
FG%            0
3P             0
3PA            0
3P%            0
2P             0
2PA            0
2P%            0
eFG%           0
FT             0
FTA            0
FT%            0
ORB            0
DRB            0
TRB            0
AST            0
STL            0
BLK            0
TOV            0
PF             0
PTS            0
Salary       382
Year           0
id             0
SalaryCap      0
dtype: int64

In [157]:
## Save dataframe to CSV
path = '/Users/takeshisugiyama/Galvanize/projects/nba-salary-prediction'
df.to_csv(path+'/scrapped_data/rawdata2000-2019.csv')

---

## Cleaning Data

In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9310 entries, 0 to 9309
Data columns (total 33 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Player     9310 non-null   object
 1   Pos        9310 non-null   object
 2   Age        9310 non-null   object
 3   Tm         9310 non-null   object
 4   G          9310 non-null   object
 5   GS         9310 non-null   object
 6   MP         9310 non-null   object
 7   FG         9310 non-null   object
 8   FGA        9310 non-null   object
 9   FG%        9310 non-null   object
 10  3P         9310 non-null   object
 11  3PA        9310 non-null   object
 12  3P%        9310 non-null   object
 13  2P         9310 non-null   object
 14  2PA        9310 non-null   object
 15  2P%        9310 non-null   object
 16  eFG%       9310 non-null   object
 17  FT         9310 non-null   object
 18  FTA        9310 non-null   object
 19  FT%        9310 non-null   object
 20  ORB        9310 non-null   obj

In [163]:
df

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,AST,STL,BLK,TOV,PF,PTS,Salary,Year,id,SalaryCap
0,TariqAbdul-Wahad,SG,25,TOT,61,56,25.9,4.5,10.6,.424,...,1.6,1.0,0.5,1.7,2.4,11.4,"$1,594,920",2000,TariqAbdul-Wahad2000,"$34,000,000"
1,ShareefAbdur-Rahim,SF,23,VAN,82,82,39.3,7.2,15.6,.465,...,3.3,1.1,1.1,3.0,3.0,20.3,"$9,000,000",2000,ShareefAbdur-Rahim2000,"$34,000,000"
2,CoryAlexander,PG,26,DEN,29,2,11.3,1.0,3.4,.286,...,2.0,0.8,0.1,1.0,1.3,2.8,"$1,980,000",2000,CoryAlexander2000,"$34,000,000"
3,RayAllen,SG,24,MIL,82,82,37.4,7.8,17.2,.455,...,3.8,1.3,0.2,2.2,2.3,22.1,"$9,000,000",2000,RayAllen2000,"$34,000,000"
4,RaferAlston,PG,23,MIL,27,0,13.4,1.0,3.5,.284,...,2.6,0.4,0.0,1.1,1.1,2.2,"$301,875",2000,RaferAlston2000,"$34,000,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9305,TraeYoung,PG,20,ATL,81,81,30.9,6.5,15.5,.418,...,8.1,0.9,0.2,3.8,1.7,19.1,"$5,363,280",2019,TraeYoung2019,"$101,869,000"
9306,CodyZeller,C,26,CHO,49,47,25.4,3.9,7.0,.551,...,2.1,0.8,0.8,1.3,3.3,10.1,"$13,528,090",2019,CodyZeller2019,"$101,869,000"
9307,TylerZeller,C,29,TOT,6,1,15.5,2.7,5.0,.533,...,0.7,0.2,0.5,0.7,3.3,7.7,"$213,948",2019,TylerZeller2019,"$101,869,000"
9308,AnteZizic,C,22,CLE,59,25,18.3,3.1,5.6,.553,...,0.9,0.2,0.4,1.0,1.9,7.8,"$1,952,760",2019,AnteZizic2019,"$101,869,000"


In [164]:
cols = df.columns.tolist()

In [182]:
cols

['id',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'Salary',
 'Year',
 'SalaryCap']

In [174]:
cols = ['id','Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
 '3PA','3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 
 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Salary', 'Year', 'SalaryCap']

In [176]:
df = df[cols]

In [181]:
cols

['id',
 'Player',
 'Pos',
 'Age',
 'Tm',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'Salary',
 'Year',
 'SalaryCap']

In [185]:
df

,id,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Salary,Year,SalaryCap
0,TariqAbdul-Wahad2000,TariqAbdul-Wahad,SG,25,TOT,61,56,25.9,4.5,10.6,.424,0.0,0.4,.130,4.4,10.2,.435,.426,2.4,3.2,.756,1.7,3.1,4.8,1.6,1.0,0.5,1.7,2.4,11.4,"$1,594,920",2000,"$34,000,000"
1,ShareefAbdur-Rahim2000,ShareefAbdur-Rahim,SF,23,VAN,82,82,39.3,7.2,15.6,.465,0.4,1.2,.302,6.9,14.4,.478,.477,5.4,6.7,.809,2.7,7.4,10.1,3.3,1.1,1.1,3.0,3.0,20.3,"$9,000,000",2000,"$34,000,000"
2,CoryAlexander2000,CoryAlexander,PG,26,DEN,29,2,11.3,1.0,3.4,.286,0.3,1.2,.257,0.7,2.2,.302,.332,0.6,0.8,.773,0.3,1.2,1.4,2.0,0.8,0.1,1.0,1.3,2.8,"$1,980,000",2000,"$34,000,000"
3,RayAllen2000,RayAllen,SG,24,MIL,82,82,37.4,7.8,17.2,.455,2.1,5.0,.423,5.7,12.2,.468,.516,4.3,4.9,.887,1.0,3.4,4.4,3.8,1.3,0.2,2.2,2.3,22.1,"$9,000,000",2000,"$34,000,000"
4,RaferAlston2000,RaferAlston,PG,23,MIL,27,0,13.4,1.0,3.5,.284,0.1,0.5,.214,0.9,3.0,.296,.300,0.1,0.1,.750,0.2,0.7,0.9,2.6,0.4,0.0,1.1,1.1,2.2,"$301,875",2000,"$34,000,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9305,TraeYoung2019,TraeYoung,PG,20,ATL,81,81,30.9,6.5,15.5,.418,1.9,6.0,.324,4.6,9.6,.477,.480,4.2,5.1,.829,0.8,2.9,3.7,8.1,0.9,0.2,3.8,1.7,19.1,"$5,363,280",2019,"$101,869,000"
9306,CodyZeller2019,CodyZeller,C,26,CHO,49,47,25.4,3.9,7.0,.551,0.1,0.4,.273,3.8,6.6,.570,.559,2.3,2.9,.787,2.2,4.6,6.8,2.1,0.8,0.8,1.3,3.3,10.1,"$13,528,090",2019,"$101,869,000"
9307,TylerZeller2019,TylerZeller,C,29,TOT,6,1,15.5,2.7,5.0,.533,0.0,0.2,.000,2.7,4.8,.552,.533,2.3,3.0,.778,1.8,2.2,4.0,0.7,0.2,0.5,0.7,3.3,7.7,"$213,948",2019,"$101,869,000"
9308,AnteZizic2019,AnteZizic,C,22,CLE,59,25,18.3,3.1,5.6,.553,0.0,0.0,,3.1,5.6,.553,.553,1.6,2.2,.705,1.8,3.6,5.4,0.9,0.2,0.4,1.0,1.9,7.8,"$1,952,760",2019,"$101,869,000"


In [189]:
numcols = ['Age',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'Salary',
 'Year',
 'SalaryCap']

In [192]:
df['Salary'] = df['Salary'].str.replace(',', '')
df['Salary'] = df['Salary'].str.replace('$', '')
df['SalaryCap'] = df['SalaryCap'].str.replace(',', '')
df['SalaryCap'] = df['SalaryCap'].str.replace('$', '')

In [197]:
df[numcols] = df[numcols].apply(pd.to_numeric)

## Split to Dataset to Train and Test (Validation) Set

In [214]:
test_set = df[df['Year'] == 2019].reset_index(drop=True)
train_set = df[df['Year'] != 2019].reset_index(drop=True)

In [217]:
path = '/Users/takeshisugiyama/Galvanize/projects/nba-salary-prediction'

In [218]:
test_set.to_csv(path+'/data/test_set.csv')
train_set.to_csv(path+'/data/train_set.csv')

### Filling NaN Salary on Test_set

Test set had 12 players that failed to merge salaries because of naming differences from the source. 

I've decided to manually input the salary on the test_set.csv dataset from the source below. 

https://hoopshype.com/salaries/players/2018-2019/

In [231]:
nosal = test_set['Salary'].isnull()
test_set[nosal]

,id,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Salary,Year,SalaryCap
31,JJBarea2019,JJBarea,PG,34,DAL,38,0,19.8,4.2,10.1,0.418,1.0,3.4,0.297,3.2,6.8,0.479,0.468,1.4,2.1,0.705,0.3,2.2,2.5,5.6,0.6,0.0,1.9,1.3,10.9,NaN,2019,101869000
151,VinceEdwards2019,VinceEdwards,SF,22,HOU,2,0,8.0,0.5,2.0,0.250,0.5,2.0,0.250,0.0,0.0,NaN,0.375,0.0,0.0,NaN,0.5,0.5,1.0,0.0,0.0,0.0,0.0,0.0,1.5,NaN,2019,101869000
206,MauriceHarkless2019,MauriceHarkless,SF,25,POR,60,53,23.6,3.2,6.5,0.487,0.6,2.0,0.275,2.6,4.5,0.582,0.530,0.8,1.2,0.671,1.3,3.2,4.5,1.2,1.1,0.9,0.8,2.7,7.7,NaN,2019,101869000
298,WaltLemon2019,WaltLemon,PG,26,CHI,6,3,27.8,6.3,14.5,0.437,0.3,0.8,0.400,6.0,13.7,0.439,0.448,1.3,1.8,0.727,0.7,3.8,4.5,5.0,1.8,0.2,1.7,2.3,14.3,NaN,2019,101869000
315,TimotheLuwawu-Cabarrot2019,TimotheLuwawu-Cabarrot,SF,23,TOT,50,7,13.4,1.6,4.4,0.376,0.7,2.3,0.310,0.9,2.0,0.451,0.459,0.6,0.8,0.756,0.2,1.7,1.9,0.5,0.4,0.2,0.4,1.5,4.6,NaN,2019,101869000
349,PattyMills2019,PattyMills,PG,30,SAS,82,1,23.3,3.4,8.1,0.425,1.9,4.9,0.394,1.5,3.2,0.475,0.545,1.1,1.3,0.854,0.3,1.9,2.2,3.0,0.6,0.1,1.1,1.6,9.9,NaN,2019,101869000
354,NazMitrou-Long2019,NazMitrou-Long,SG,25,UTA,14,0,6.0,0.4,1.4,0.300,0.1,0.8,0.182,0.3,0.6,0.444,0.350,0.1,0.1,1.000,0.1,0.4,0.4,1.1,0.1,0.1,0.6,0.7,1.1,NaN,2019,101869000
418,CameronReynolds2019,CameronReynolds,SG,23,MIN,19,0,13.6,1.7,4.1,0.423,1.1,2.7,0.412,0.6,1.4,0.444,0.558,0.4,0.5,0.889,0.2,1.4,1.6,0.7,0.3,0.1,0.3,1.4,5.0,NaN,2019,101869000
438,DennisSchroder2019,DennisSchroder,PG,25,OKC,79,14,29.3,5.8,14.0,0.414,1.6,4.6,0.341,4.2,9.4,0.450,0.470,2.4,2.9,0.819,0.5,3.1,3.6,4.1,0.8,0.2,2.2,2.4,15.5,NaN,2019,101869000
453,IshSmith2019,IshSmith,PG,30,DET,56,0,22.3,3.7,8.7,0.419,0.8,2.4,0.326,2.9,6.3,0.455,0.464,0.8,1.1,0.758,0.4,2.2,2.6,3.6,0.5,0.2,1.1,1.9,8.9,NaN,2019,101869000
